In [47]:
import pandas as pd
import numpy as np

import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
np.random.seed(42)

from matplotlib import style
style.use('fivethirtyeight')

In [50]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [91]:
df_test

,image
0,images/18353.jpg
1,images/18354.jpg
2,images/18355.jpg
3,images/18356.jpg
4,images/18357.jpg
...,...
8795,images/27148.jpg
8796,images/27149.jpg
8797,images/27150.jpg
8798,images/27151.jpg


In [52]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18353 entries, 0 to 18352
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   18353 non-null  object
 1   label   18353 non-null  object
dtypes: object(2)
memory usage: 286.9+ KB


In [53]:
column = 'label'
if df_train['label'].dtype == 'object' :
    
    #print(type(df[column]))
    value_counts = df_train[column].value_counts()
    num_categories = len(value_counts)

    # İki alt kategorisi varsa 0 ve 1 olarak kodlayalım
    if num_categories == 2:
        category_mapping = {category: index for index, category in enumerate(value_counts.index)}
        df_train['lab'] = df_train[column].map(category_mapping)
    # İki alt kategoriden fazlaysa 1'den başlayan sıralı sayılarla kodlayalım
    else:
        category_mapping = {category: index + 1 for index, category in enumerate(value_counts.index)}
        df_train['lab'] = df_train[column].map(category_mapping)

In [54]:
list_number = df_train['lab'].value_counts().index


In [55]:
list_number

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            167, 168, 169, 170, 171, 172, 173, 174, 175, 176],
           dtype='int64', length=176)

In [56]:
value_counts=df_train['label'].value_counts().index.tolist()

In [57]:
df_list = pd.DataFrame(value_counts, columns=['column_name'])

In [58]:
df_list['index'] = list_number

In [59]:
df_list

,column_name,index
0,maclura_pomifera,1
1,ulmus_rubra,2
2,prunus_virginiana,3
3,acer_rubrum,4
4,broussonettia_papyrifera,5
...,...,...
171,cedrus_deodara,172
172,ailanthus_altissima,173
173,crataegus_crus-galli,174
174,evodia_daniellii,175


In [60]:
images_array = []
label = []
str_path = "C:/Users/haluk/Documents/AI_/Become a Pro/Leave Classification/"
rotate_angle = 20  # Define the rotation angle in degrees

for i in range(len(df_train)):
    path = df_train['image'].loc[i]
    image = Image.open(str_path + path)
    
    # Resize the image to (60, 60)
    resize_img = image.resize((60, 60))
    #add image to array list
    images_array.append(np.array(resize_img))
    label.append(df_train['lab'].loc[i])
        
    
    # Rotate the image by 45 degrees
    #rotated_image = resize_img.rotate(rotate_angle, resample=Image.BILINEAR, expand=True)
    resize_img_r = image.resize((80, 80))
    rotated_image = resize_img_r.rotate(rotate_angle, resample=Image.BILINEAR, expand=True)
    resize_img_rot = rotated_image.resize((60, 60))
    #add image to array list
    images_array.append(np.array(resize_img_rot))
    label.append(df_train['lab'].loc[i])

In [61]:
import cv2

images_array_t = []
label_t = []
#str_path = "C:/Users/haluk/Documents/AI_/Specialze in Data Science 21Project/New folder/"
str_path = "C:/Users/haluk/Documents/AI_/Become a Pro/Leave Classification/"
#for path in df_train['Path']:
for i in range(len(df_test)):
    path = df_test['image'].loc[i]   
    #print(path)
    image = cv2.imread(path)
    #print(image)
    img_array = Image.fromarray(image,'RGB')
    resize_img = img_array.resize((60,60))
    images_array_t.append(np.array(resize_img))
    #label_t.append(df_test['ClassId'].loc[i])

In [62]:
len(images_array_t)

8800

In [63]:
## number of categories
len(df_train['lab'].unique())

176

In [64]:
#convert list to array for further processing
# 
X = np.array(images_array)
Y = np.array(label)

X_t = np.array(images_array_t)
#Y_t = np.array(label_t)


In [65]:
X.shape, Y.shape

((36706, 60, 60, 3), (36706,))

In [66]:
#normalization 
X = X/255;
X_t = X_t /255;

In [67]:
#from tensorflow.keras.utils import to_categorical

# Convert y_train to one-hot encoded format
#Y = to_categorical(Y, num_classes=176)

In [68]:
# split the data set as train and test
#from sklearn.model_selection import train_test_split
#x_train, x_test , y_train,y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

In [69]:
'''
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Assuming y contains your original integer labels
num_classes = 177  # Number of classes

# Split your dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# One-hot encode the labels

y_train_encoded = tf.keras.utils.to_categorical(y_train,num_classes)
y_test_encoded = tf.keras.utils.to_categorical(y_test,num_classes)
'''

'\nimport tensorflow as tf\nfrom sklearn.model_selection import train_test_split\n\n# Assuming y contains your original integer labels\nnum_classes = 177  # Number of classes\n\n# Split your dataset into training and testing sets\nx_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)\n\n# One-hot encode the labels\n\ny_train_encoded = tf.keras.utils.to_categorical(y_train,num_classes)\ny_test_encoded = tf.keras.utils.to_categorical(y_test,num_classes)\n'

In [76]:
num_classes = 177
x_train = X;
y_train_encoded = tf.keras.utils.to_categorical(Y,num_classes);
x_test = X_t;

In [77]:
x_train.shape, y_train_encoded.shape

((36706, 60, 60, 3), (36706, 177))

In [78]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the CNN model
model = models.Sequential()

# Convolutional layers
# input layer size is equal to frame pixel size
model.add(layers.Conv2D(32, (4, 4), activation='relu', input_shape=(60, 60, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(200, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output for fully connected layers
model.add(layers.Flatten())

# Fully connected layers
model.add(layers.Dense(264, activation='relu'))
model.add(layers.Dense(177, activation='softmax')) 

In [79]:
model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy'])

In [80]:
history = model.fit(x_train, y_train_encoded, batch_size=20, epochs=50, validation_split=0.1)

Epoch 1/50
1652/1652 [==============================] - 118s 71ms/step - loss: 0.0330 - accuracy: 0.0665 - val_loss: 0.0266 - val_accuracy: 0.1373
Epoch 2/50
1652/1652 [==============================] - 124s 75ms/step - loss: 0.0224 - accuracy: 0.2437 - val_loss: 0.0209 - val_accuracy: 0.2800
Epoch 3/50
1652/1652 [==============================] - 140s 85ms/step - loss: 0.0179 - accuracy: 0.3899 - val_loss: 0.0174 - val_accuracy: 0.3977
Epoch 4/50
1652/1652 [==============================] - 141s 85ms/step - loss: 0.0147 - accuracy: 0.5028 - val_loss: 0.0157 - val_accuracy: 0.4751
Epoch 5/50
1652/1652 [==============================] - 141s 85ms/step - loss: 0.0124 - accuracy: 0.5846 - val_loss: 0.0151 - val_accuracy: 0.5086
Epoch 6/50
1652/1652 [==============================] - 141s 85ms/step - loss: 0.0107 - accuracy: 0.6492 - val_loss: 0.0149 - val_accuracy: 0.5203
Epoch 7/50
1652/1652 [==============================] - 141s 85ms/step - loss: 0.0094 - accuracy: 0.7023 - val_loss: 0

In [81]:
prediction = model.predict(x_test)

275/275 [==============================] - 7s 26ms/step


In [82]:
prediction.shape

(8800, 177)

In [89]:
pre_indx  = []
leave_type = []
for pr in prediction :
    pre_indx.append( np.argmax(pr))
    leave_type.append( df_list.at[int(np.argmax(pr))-1, 'column_name'] )
    
    

In [92]:
df_test['label'] = leave_type

In [93]:
df_test

,image,label
0,images/18353.jpg,quercus_phellos
1,images/18354.jpg,prunus_virginiana
2,images/18355.jpg,quercus_velutina
3,images/18356.jpg,pinus_bungeana
4,images/18357.jpg,prunus_sargentii
...,...,...
8795,images/27148.jpg,pinus_resinosa
8796,images/27149.jpg,ilex_opaca
8797,images/27150.jpg,pinus_resinosa
8798,images/27151.jpg,juniperus_virginiana


In [94]:
df_test.to_csv("submit_set.csv",index=False)